## Competição DSA 2019/03
<h3>Prever vendas das Farmácias Solar</h3>

Fazendo as importações necessárias (bibliotecas e arquivos)

In [65]:
from sklearn.model_selection import ShuffleSplit

In [60]:
# Importar as bibliotecas necessárias para este notebook
from datetime                import datetime
from sklearn                 import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble        import GradientBoostingRegressor
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from sklearn.datasets        import load_digits
from sklearn.model_selection import learning_curve
# from sklearn                 import cross_validation
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

In [2]:
# importar os arquivos para dataframes:
train_file = pd.read_csv('dataset_treino.csv')
test_file  = pd.read_csv('dataset_teste.csv')
store_file = pd.read_csv('lojas.csv')

/home/angelo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Entender os dados

In [3]:
# Visualiar primeiras linhas dos arquivos:
train_file.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [5]:
test_file.head(10)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
5,6,10,4,2015-09-17,1.0,1,0,0
6,7,11,4,2015-09-17,1.0,1,0,0
7,8,12,4,2015-09-17,1.0,1,0,0
8,9,13,4,2015-09-17,1.0,1,0,0
9,10,14,4,2015-09-17,1.0,1,0,0


In [6]:
test_file.Date.describe()

count          41088
unique            48
top       2015-09-03
freq             856
Name: Date, dtype: object

In [7]:
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
Id               41088 non-null int64
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Date             41088 non-null object
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null int64
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


In [8]:
store_file.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [9]:
store_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1112 non-null float64
CompetitionOpenSinceMonth    761 non-null float64
CompetitionOpenSinceYear     761 non-null float64
Promo2                       1115 non-null int64
Promo2SinceWeek              571 non-null float64
Promo2SinceYear              571 non-null float64
PromoInterval                571 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


Tratar valores NaN

In [10]:
train_file.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [11]:
test_file.isna().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [12]:
store_file.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [13]:
# DataFrame de treino possui a variável Customers
# estas não estão presentes no dataFrame de testes, assim, serão excluídas
train_file.drop(['Customers'], axis=1, inplace = True); train_file.head()

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,1,1,0,1
1,2,5,2015-07-31,6064,1,1,0,1
2,3,5,2015-07-31,8314,1,1,0,1
3,4,5,2015-07-31,13995,1,1,0,1
4,5,5,2015-07-31,4822,1,1,0,1


In [14]:
# DataFrame test_file possui a coluna Open (indicador loja aberta: 0 = fechada, 1 = aberta)
# Substituir NaN por 0
test_file.Open.fillna(0, inplace = True); test_file.isna().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [15]:
# DataFrame store_file possui diversas colunas NaN. Entender caso a caso.
# Primeiro, e menor caso, CompetitionDistance com apenas 3 ocorrências,
# substituir pela média
store_file.CompetitionDistance.fillna(store_file.CompetitionDistance.mean(), inplace = True); store_file.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [16]:
# Segundo caso, variável CompetitionOpenSinceMonth (mês aproximado em que o concorrente mais próximo foi aberto)
print(store_file.CompetitionOpenSinceMonth.describe())
# Substituir os valores NaN pelo mínimo
store_file.CompetitionOpenSinceMonth.fillna(store_file.CompetitionOpenSinceMonth.min(), inplace = True); store_file.isna().sum()

count    761.000000
mean       7.224704
std        3.212348
min        1.000000
25%        4.000000
50%        8.000000
75%       10.000000
max       12.000000
Name: CompetitionOpenSinceMonth, dtype: float64


Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth      0
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [17]:
# Terceiro caso, variável CompetitionOpenSinceYear
print(store_file.CompetitionOpenSinceYear.describe())
# Substituir NaN pelo mínimo
store_file.CompetitionOpenSinceYear.fillna(store_file.CompetitionOpenSinceYear.min(), inplace = True); store_file.isna().sum()

count     761.000000
mean     2008.668857
std         6.195983
min      1900.000000
25%      2006.000000
50%      2010.000000
75%      2013.000000
max      2015.000000
Name: CompetitionOpenSinceYear, dtype: float64


Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth      0
CompetitionOpenSinceYear       0
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [18]:
# Quarto caso, variável Promo2SinceWeek (descreve a semana do calendário em que a loja começou a participar do Promo2)
print(store_file.Promo2SinceWeek.describe())
print(store_file.Promo2SinceWeek.unique())
# Esta variável tem relação com a Promo2 (promoção contínua: 0 = loja não está participando, 1 = loja está participando)
# Identificar o range de valores de Promo2 onde Promo2SinceWeek é NaN
store_file[store_file['Promo2SinceWeek'].isnull()]['Promo2'].describe()
# Substituir NaN por zero
store_file.Promo2SinceWeek.fillna(0, inplace = True); store_file.isna().sum()

count    571.000000
mean      23.595447
std       14.141984
min        1.000000
25%       13.000000
50%       22.000000
75%       37.000000
max       50.000000
Name: Promo2SinceWeek, dtype: float64
[nan 13. 14.  1. 45. 40. 26. 22.  5.  6. 10. 31. 37.  9. 39. 27. 18. 35.
 23. 48. 36. 50. 44. 49. 28.]


Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth      0
CompetitionOpenSinceYear       0
Promo2                         0
Promo2SinceWeek                0
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [19]:
# Quinto caso, variável Promo2SinceYear (descreve o ano do calendário em que a loja começou a participar do Promo2)
print(store_file.Promo2SinceYear.describe())
print(store_file.Promo2SinceYear.unique())
# Esta variável tem relação com a Promo2 (promoção contínua: 0 = loja não está participando, 1 = loja está participando)
# Identificar o range de valores de Promo2 onde Promo2SinceWeek é NaN
store_file[store_file['Promo2SinceYear'].isnull()]['Promo2'].describe()
# Substituir NaN por zero
store_file.Promo2SinceYear.fillna(0, inplace = True); store_file.isna().sum()

count     571.000000
mean     2011.763573
std         1.674935
min      2009.000000
25%      2011.000000
50%      2012.000000
75%      2013.000000
max      2015.000000
Name: Promo2SinceYear, dtype: float64
[  nan 2010. 2011. 2012. 2009. 2014. 2015. 2013.]


Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth      0
CompetitionOpenSinceYear       0
Promo2                         0
Promo2SinceWeek                0
Promo2SinceYear                0
PromoInterval                544
dtype: int64

In [20]:
# Sexto caso, variável PromoInterval (intervalos consecutivos onde Promo2 é iniciado)
print(store_file.PromoInterval.describe())
print(store_file.PromoInterval.unique())
# Esta variável tem relação com a Promo2 (promoção contínua: 0 = loja não está participando, 1 = loja está participando)
# Identificar o range de valores de Promo2 onde Promo2SinceWeek é NaN
store_file[store_file['PromoInterval'].isnull()]['Promo2'].describe()
# Substituir NaN por zero
store_file.PromoInterval.fillna(0, inplace = True); store_file.isna().sum()

count                 571
unique                  3
top       Jan,Apr,Jul,Oct
freq                  335
Name: PromoInterval, dtype: object
[nan 'Jan,Apr,Jul,Oct' 'Feb,May,Aug,Nov' 'Mar,Jun,Sept,Dec']


Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [21]:
# Alterar variáveis para int
store_file.CompetitionOpenSinceMonth = store_file.CompetitionOpenSinceMonth.astype('int64')
store_file.CompetitionOpenSinceYear  = store_file.CompetitionOpenSinceYear.astype('int64')
store_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1115 non-null float64
CompetitionOpenSinceMonth    1115 non-null int64
CompetitionOpenSinceYear     1115 non-null int64
Promo2                       1115 non-null int64
Promo2SinceWeek              1115 non-null float64
Promo2SinceYear              1115 non-null float64
PromoInterval                1115 non-null object
dtypes: float64(3), int64(4), object(3)
memory usage: 87.2+ KB


In [22]:
# Criar variável do tipo datetime
store_file['CompetitionOpenYMD'] = pd.to_datetime(store_file.CompetitionOpenSinceYear.astype(str) + '/' + store_file.CompetitionOpenSinceMonth.astype(str) + '/01')

In [23]:
# Com isso, a variável CompetitionOpenSinceMonth pode ser apagada
store_file.drop(['CompetitionOpenSinceMonth'], axis=1, inplace = True)

In [24]:
store_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                       1115 non-null int64
StoreType                   1115 non-null object
Assortment                  1115 non-null object
CompetitionDistance         1115 non-null float64
CompetitionOpenSinceYear    1115 non-null int64
Promo2                      1115 non-null int64
Promo2SinceWeek             1115 non-null float64
Promo2SinceYear             1115 non-null float64
PromoInterval               1115 non-null object
CompetitionOpenYMD          1115 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 87.2+ KB


In [25]:
store_file.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD
0,1,c,a,1270.0,2008,0,0.0,0.0,0,2008-09-01
1,2,a,a,570.0,2007,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2007-11-01
2,3,a,a,14130.0,2006,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2006-12-01
3,4,c,c,620.0,2009,0,0.0,0.0,0,2009-09-01
4,5,a,a,29910.0,2015,0,0.0,0.0,0,2015-04-01


In [26]:
## Alterar os valores de StoreType e Assortment para dummies Pandas
store_file = pd.get_dummies(store_file, columns = ['StoreType', 'Assortment'])
print(store_file.info())
store_file.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 15 columns):
Store                       1115 non-null int64
CompetitionDistance         1115 non-null float64
CompetitionOpenSinceYear    1115 non-null int64
Promo2                      1115 non-null int64
Promo2SinceWeek             1115 non-null float64
Promo2SinceYear             1115 non-null float64
PromoInterval               1115 non-null object
CompetitionOpenYMD          1115 non-null datetime64[ns]
StoreType_a                 1115 non-null uint8
StoreType_b                 1115 non-null uint8
StoreType_c                 1115 non-null uint8
StoreType_d                 1115 non-null uint8
Assortment_a                1115 non-null uint8
Assortment_b                1115 non-null uint8
Assortment_c                1115 non-null uint8
dtypes: datetime64[ns](1), float64(3), int64(3), object(1), uint8(7)
memory usage: 77.4+ KB
None


,Store,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
0,1,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0
1,2,570.0,2007,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2007-11-01,1,0,0,0,1,0,0
2,3,14130.0,2006,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2006-12-01,1,0,0,0,1,0,0
3,4,620.0,2009,0,0.0,0.0,0,2009-09-01,0,0,1,0,0,0,1
4,5,29910.0,2015,0,0.0,0.0,0,2015-04-01,1,0,0,0,1,0,0


In [27]:
store_file.isna().sum()

Store                       0
CompetitionDistance         0
CompetitionOpenSinceYear    0
Promo2                      0
Promo2SinceWeek             0
Promo2SinceYear             0
PromoInterval               0
CompetitionOpenYMD          0
StoreType_a                 0
StoreType_b                 0
StoreType_c                 0
StoreType_d                 0
Assortment_a                0
Assortment_b                0
Assortment_c                0
dtype: int64

In [28]:
train_file.head()

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,1,1,0,1
1,2,5,2015-07-31,6064,1,1,0,1
2,3,5,2015-07-31,8314,1,1,0,1
3,4,5,2015-07-31,13995,1,1,0,1
4,5,5,2015-07-31,4822,1,1,0,1


In [29]:
# Criar variável do tipo datetime nos dataframes train_file e test_file
train_file.Date = pd.to_datetime(train_file['Date'], format = "%Y-%m-%d")
test_file.Date  = pd.to_datetime(test_file['Date'], format = "%Y-%m-%d")
# Checar os resultados:
print(train_file["Date"].head())
print(test_file["Date"].head())

0   2015-07-31
1   2015-07-31
2   2015-07-31
3   2015-07-31
4   2015-07-31
Name: Date, dtype: datetime64[ns]
0   2015-09-17
1   2015-09-17
2   2015-09-17
3   2015-09-17
4   2015-09-17
Name: Date, dtype: datetime64[ns]


In [30]:
# Alterar a coluna StateHoliday para dummies
train_file = pd.get_dummies(train_file, columns = ['StateHoliday'])
test_file = pd.get_dummies(test_file, columns = ['StateHoliday'])

In [31]:
test_file.head()

,Id,Store,DayOfWeek,Date,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a
0,1,1,4,2015-09-17,1.0,1,0,1,0
1,2,3,4,2015-09-17,1.0,1,0,1,0
2,3,7,4,2015-09-17,1.0,1,0,1,0
3,4,8,4,2015-09-17,1.0,1,0,1,0
4,5,9,4,2015-09-17,1.0,1,0,1,0


In [32]:
train_file.head()

,Store,DayOfWeek,Date,Sales,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,1,5,2015-07-31,5263,1,1,1,0,1,0,0,0
1,2,5,2015-07-31,6064,1,1,1,0,1,0,0,0
2,3,5,2015-07-31,8314,1,1,1,0,1,0,0,0
3,4,5,2015-07-31,13995,1,1,1,0,1,0,0,0
4,5,5,2015-07-31,4822,1,1,1,0,1,0,0,0


In [33]:
# Juntar os dados de store com train_file e test_file
df_train = pd.merge(train_file, store_file, on = 'Store')
df_test  = pd.merge(test_file, store_file,  on = 'Store')

In [34]:
# A função abaixo é de autoria de Lukas e Rich Andrews, usuários do stackoverflow. A eles meu muito obrigado
# Lukas: https://stackoverflow.com/users/10257810/lukas; Rich https://stackoverflow.com/users/9324678/rich-andrews
def matched(row):
    if type(row['PromoInterval'])==str:
        return row['Date'].month in [datetime.strptime(mon.strip()[:3], '%b').month for mon in row['PromoInterval'].split(',')]        
    else:
        # for numbers - return True if months match
        return row['Date'].month==row['PromoInterval']

In [35]:
# Aplicar a função aos dataFrames
df_train['IsItInPromotion'] = df_train.apply(matched, axis=1).astype(int)
df_test['IsItInPromotion']  = df_test.apply(matched, axis=1).astype(int)

In [36]:
pd.set_option('display.max_columns', 60)
df_train.head(15)

,Store,DayOfWeek,Date,Sales,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,IsItInPromotion
0,1,5,2015-07-31,5263,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
1,1,4,2015-07-30,5020,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
2,1,3,2015-07-29,4782,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
3,1,2,2015-07-28,5011,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
4,1,1,2015-07-27,6102,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
5,1,7,2015-07-26,0,0,0,0,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
6,1,6,2015-07-25,4364,1,0,0,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
7,1,5,2015-07-24,3706,1,0,0,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
8,1,4,2015-07-23,3769,1,0,0,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
9,1,3,2015-07-22,3464,1,0,0,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0


In [37]:
df_test.head(15)

,Id,Store,DayOfWeek,Date,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,IsItInPromotion
0,1,1,4,2015-09-17,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
1,857,1,3,2015-09-16,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
2,1713,1,2,2015-09-15,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
3,2569,1,1,2015-09-14,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
4,3425,1,7,2015-09-13,0.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
5,4281,1,6,2015-09-12,1.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
6,5137,1,5,2015-09-11,1.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
7,5993,1,4,2015-09-10,1.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
8,6849,1,3,2015-09-09,1.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0
9,7705,1,2,2015-09-08,1.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0


In [38]:
# Criar coluna com diferença entre as datas entre Date e CompetitionOpenYMD
df_train['DifferenceBetweenDates'] = df_train.Date - df_train.CompetitionOpenYMD
df_test['DifferenceBetweenDates']  = df_test.Date  - df_test.CompetitionOpenYMD

In [39]:
# Criar indicadores boolean a partir da coluna Date: IsMonthFinal, IsMonthBegin, IsYearFinal e IsYearBegin
# dataFrame de treino
# Criar indicadores boolean a partir da coluna Date: IsMonthFinal, IsMonthBegin, IsYearFinal e IsYearBegin
# dataFrame de treino
df_train.loc[(df_train.Date.dt.day    > 20), 'IsMonthFinal'] = 1
df_train.loc[(df_train.Date.dt.day   <= 20), 'IsMonthFinal'] = 0
df_train.loc[(df_train.Date.dt.day    < 10), 'IsMonthBegin'] = 1
df_train.loc[(df_train.Date.dt.day   >= 10), 'IsMonthBegin'] = 0
df_train.loc[(df_train.Date.dt.month == 12), 'IsYearFinal']  = 1
df_train.loc[(df_train.Date.dt.month  < 12), 'IsYearFinal']  = 0
df_train.loc[(df_train.Date.dt.month  == 1), 'IsYearBegin']  = 1
df_train.loc[(df_train.Date.dt.month   > 1), 'IsYearBegin']  = 0
# dataFrame de teste
df_test.loc[(df_test.Date.dt.day    > 20), 'IsMonthFinal'] = 1
df_test.loc[(df_test.Date.dt.day   <= 20), 'IsMonthFinal'] = 0
df_test.loc[(df_test.Date.dt.day    < 10), 'IsMonthBegin'] = 1
df_test.loc[(df_test.Date.dt.day   >= 10), 'IsMonthBegin'] = 0
df_test.loc[(df_test.Date.dt.month == 12), 'IsYearFinal']  = 1
df_test.loc[(df_test.Date.dt.month  < 12), 'IsYearFinal']  = 0
df_test.loc[(df_test.Date.dt.month  == 1), 'IsYearBegin']  = 1
df_test.loc[(df_test.Date.dt.month   > 1), 'IsYearBegin']  = 0

In [40]:
# Já que temos o dia da semana em inteiros (DayOfWeek), capturar também o dia do ano
# Na ordem treino e teste:
df_train['DayOfYear'] = df_train.Date.dt.dayofyear
df_test['DayOfYear']  = df_test.Date.dt.dayofyear

In [41]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,IsItInPromotion,DifferenceBetweenDates,IsMonthFinal,IsMonthBegin,IsYearFinal,IsYearBegin,DayOfYear
0,1,5,2015-07-31,5263,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2524 days,1.0,0.0,0.0,0.0,212
1,1,4,2015-07-30,5020,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2523 days,1.0,0.0,0.0,0.0,211
2,1,3,2015-07-29,4782,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2522 days,1.0,0.0,0.0,0.0,210
3,1,2,2015-07-28,5011,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2521 days,1.0,0.0,0.0,0.0,209
4,1,1,2015-07-27,6102,1,1,1,0,1,0,0,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2520 days,1.0,0.0,0.0,0.0,208


In [42]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenYMD,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,IsItInPromotion,DifferenceBetweenDates,IsMonthFinal,IsMonthBegin,IsYearFinal,IsYearBegin,DayOfYear
0,1,1,4,2015-09-17,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2572 days,0.0,0.0,0.0,0.0,260
1,857,1,3,2015-09-16,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2571 days,0.0,0.0,0.0,0.0,259
2,1713,1,2,2015-09-15,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2570 days,0.0,0.0,0.0,0.0,258
3,2569,1,1,2015-09-14,1.0,1,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2569 days,0.0,0.0,0.0,0.0,257
4,3425,1,7,2015-09-13,0.0,0,0,1,0,1270.0,2008,0,0.0,0.0,0,2008-09-01,0,0,1,0,1,0,0,0,2568 days,0.0,0.0,0.0,0.0,256


In [43]:
# Com isso, a coluna CompetitionOpenYMD podem ser apagada
df_train.drop(['CompetitionOpenYMD'], axis=1, inplace = True)
df_test.drop(['CompetitionOpenYMD'], axis=1, inplace = True)

In [44]:
# Alterar a variável DifferenceBetweenDates de Timedelta para int32
df_train.DifferenceBetweenDates = df_train.DifferenceBetweenDates.dt.days.astype('int32')
df_test.DifferenceBetweenDates = df_test.DifferenceBetweenDates.dt.days.astype('int32')

In [45]:
df_train.SchoolHoliday.unique()

array([1, 0])

Dados tratados, começa a modelagem

In [46]:
# Dividir os dados em treino e teste
y = df_train.Sales
x = df_train
x.drop(['Sales', 'PromoInterval'], axis=1, inplace = True)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101)

In [47]:
# Parametrizar o modelo
# Criar instância do modelo
gbrt   = GradientBoostingRegressor(n_estimators=100)

In [48]:
# Treinar o modelo
X_train.drop(['Date'], axis=1, inplace = True)
gbrt.fit(X_train, y_train)
# X_train.info()

/home/angelo/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [50]:
# Fazer as predições
X_test.drop(['Date'], axis=1, inplace = True)
predictions = gbrt.predict(X_test)

/home/angelo/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [51]:
# Métricas encontradas
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 1524.9499609290403
MSE: 4932294.172595183
RMSE: 2220.8768927149436


<hr/>
<p><b>Linear modelo com colunas ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']</b></p>
<p>MAE: 1773.391653270394</p>
<p>MSE: 6923669.887507999</p>
<p>RMSE: 2631.2867360871182</p>
<p><b>Segunda tentativa, com todas as colunas e dados não normalizados</b></p>
<p>MAE: 1779.7226105572372</p>
<p>MSE: 6520572.7667677775</p>
<p>RMSE: 2553.541220886747</p>
<p><b>Terceira tentativa, com valores dummies e dados não normalizados</b></p>
<p>MAE: 1751.2452511394356</p>
<p>MSE: 6276716.411293483</p>
<p>RMSE: 2505.33758429747</p>
<p><b>Quarta tentativa, resultados com GBR e dados não normalizados</b></p>
<p>MAE: 1524.9499609290403</p>
<p>MSE: 4932294.172595183</p>
<p>RMSE: 2220.8768927149436</p>
<hr/>

In [53]:
# Função para tunar o modelo
def GradientBooster(param_grid, n_jobs): 
    estimator = GradientBoostingRegressor()
    cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2)
    classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs)
    classifier.fit(X_train, y_train)
    print("Best Estimator learned through GridSearch")
    print(classifier.best_estimator_)
    return cv, classifier.best_estimator_

In [61]:
# Plot para identificar a peerformance do modelo ao longo dos ciclos
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() 
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")
    return plt

In [66]:
# Executar o tuning
param_grid = {'n_estimators':[25],
              'learning_rate': [0.1],   # 0.05, 0.02, 0.01],
              'max_depth':[6],          #4,6],
              'min_samples_leaf':[3],   #,5,9,17]
              'max_features':[1.0],     #,0.3]#,0.1] 
             }
n_jobs = 3
              
cv, best_est = GradientBooster(param_grid, n_jobs)

TypeError: __init__() got an unexpected keyword argument 'n_iter'

In [68]:
# Gerar arquivo de resposta
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/angelo/anaconda3/lib/python3.7/site-packages (0.20.3)


In [75]:
# df_test.drop(['Date'], axis=1, inplace = True)
df_test.drop(['PromoInterval'], axis=1, inplace = True)

In [81]:
df_test['StateHoliday_0'] = 0
df_test['StateHoliday_b'] = 0
df_test['StateHoliday_c'] = 0

In [82]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 29 columns):
Id                          41088 non-null int64
Store                       41088 non-null int64
DayOfWeek                   41088 non-null int64
Open                        41088 non-null float64
Promo                       41088 non-null int64
SchoolHoliday               41088 non-null int64
StateHoliday_0              41088 non-null int64
StateHoliday_a              41088 non-null uint8
CompetitionDistance         41088 non-null float64
CompetitionOpenSinceYear    41088 non-null int64
Promo2                      41088 non-null int64
Promo2SinceWeek             41088 non-null float64
Promo2SinceYear             41088 non-null float64
StoreType_a                 41088 non-null uint8
StoreType_b                 41088 non-null uint8
StoreType_c                 41088 non-null uint8
StoreType_d                 41088 non-null uint8
Assortment_a                41088 non-null uint8
A

In [83]:
predict_test_file = gbrt.predict(df_test)

In [84]:
predict_test_file[predict_test_file < 0] = train_file.Sales.min()

In [85]:
# Armazenar o resultado em dataframe:
nome_colunas   = ['Id', 'Sales']
resposta       = pd.DataFrame(columns = nome_colunas)
resposta.Id    = test_file.Id
resposta.Sales = predict_test_file

In [86]:
# Gravar arquivo
resposta.to_csv('resposta.csv', index = False)